In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1)

cache_dir = LocalFileStore("./.cache/")

loader = UnstructuredFileLoader("./files/moby_dick.txt")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=300,
    chunk_overlap=50,
)
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings(
    model = "text-embedding-3-small"
)

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

chain.run("How many people show up in the story. what's there name?")

ImportError: cannot import name 'field_validator' from 'pydantic' (/Users/sugang/Desktop/projects/fullstack_gpt/venv/lib/python3.11/site-packages/pydantic/__init__.cpython-311-darwin.so)